In [1]:
import pandas

%store -r WORKDIR

if 'WORKDIR' not in dir():
    WORKDIR = 'C:/Users/BZ241WX/Documents/InfNet/CursoPosDataScience/mit_data_science/'

    
source_file = WORKDIR + '/Data/Raw/winequality-red-dev.parquet'
save_file = WORKDIR + '/Data/Processed/winequality-red-proc.parquet'


# Leitura dos Dados 

In [2]:
data = pandas.read_parquet(source_file)

print('shape:', data.shape)
print('columns:', data.columns)
data.head()

shape: (1279, 13)
columns: Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'name', 'good_quality'],
      dtype='object')


,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,name,good_quality
1033,7.5,0.570,0.08,2.6,0.089,14.0,27.0,0.99592,3.30,0.59,10.4,wine-red1033,True
193,7.6,0.550,0.21,2.2,0.071,7.0,28.0,0.99640,3.28,0.55,9.7,wine-red193,False
800,7.2,0.610,0.08,4.0,0.082,26.0,108.0,0.99641,3.25,0.51,9.4,wine-red800,False
1461,6.2,0.785,0.00,2.1,0.060,6.0,13.0,0.99664,3.59,0.61,10.0,wine-red1461,False
1086,8.5,0.340,0.40,4.7,0.055,3.0,9.0,0.99738,3.38,0.66,11.6,wine-red1086,True


# Estatistica dos dados 

In [3]:
data.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
count,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.000000,1279.00000,1279.000000
mean,8.311181,0.531196,0.269124,2.532721,0.086742,15.904613,46.719312,0.996719,3.312955,0.65552,10.436630
std,1.750285,0.180402,0.196297,1.346453,0.045411,10.442568,33.455641,0.001853,0.157363,0.16919,1.061419
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.33000,8.400000
25%,7.100000,0.397500,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.55000,9.500000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,14.000000,38.000000,0.996720,3.310000,0.62000,10.200000
75%,9.300000,0.640000,0.425000,2.600000,0.090000,21.000000,63.000000,0.997800,3.400000,0.73000,11.100000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003200,4.010000,2.00000,14.900000


# Verificar Valores Nulos

In [4]:
data.isnull().sum()

fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
name                    0
good_quality            0
dtype: int64

# Variável Alvo 

In [5]:
data.groupby('good_quality').pH.count()

good_quality
False    595
True     684
Name: pH, dtype: int64

# Exportar Base de Dados 

In [6]:
data.to_parquet(save_file)